### Import's

In [ ]:
import pandas as pd
import statistics

### Lendo arquivo csv

In [ ]:
## Ler o arquivo csv, e ler apenas as colunas ['Palavras', 'Rotulo', 'Sentenca']
df_original = pd.read_csv("./Divergencias.csv", usecols=['Palavras', 'Rotulo', 'Sentenca'])

In [71]:
display(df_original)

,Palavras,Rotulo,Sentenca
0,id,O,1
1,:,O,1
2,0,O,1
3,NOME,O,2
4,DA,O,2
...,...,...,...
186014,Destilados,O,14287
186015,do,O,14287
186016,Brasil,B-NOME_LOCAL,14287
186017,2019,B-TEMPO,14287


### Filtrando e pegando as informações de instâncias

In [79]:
index_remove = []
lista_tokens_inst = [] ## Lista para armazenar tokens por instâncias
lista_sentencas_inst = [[0,0]] ## Lista para armazenar tokens por instâncias, com valor presetado (o padrão é [<sentença atual>,<qtd de senteças>])
cont_tokens = 0
for index, dado in df_original.iterrows():
    if (len(dado['Palavras']) == 1 and not dado['Palavras'][0].isalnum()) and dado["Rotulo"] == 'O': ## Retirar os caracteres especiais que estão sozinhos
        index_remove.append(index)
    elif (dado['Palavras'] == 'id'): ## Retirar o id e o número que o acompanha
        index_remove.append(index)
        index_remove.append(index + 2) ## Para pegar o número do id
        lista_tokens_inst.append(cont_tokens - 1) ## Adiciona os token's calculados do id anterior na lista, o -1 é para retirar da contagem o token do número do id
        cont_tokens = 0 ## Zera o contador para iniciar uma nova contagem de tokens por instância
        lista_sentencas_inst[-1][1] -= 1 ## Reduz uma sentença visto que o o valor id é contado como sentença
        lista_sentencas_inst.append([0,0]) ## Inicio outra contagem de sentença por instância
    else:
        if dado['Sentenca'] != lista_sentencas_inst[-1][0]: ## Vejo se estou mexendo com outra sentença, caso se atualizo a sentença atual e adiciono um ao contador
            lista_sentencas_inst[-1][0] = dado['Sentenca']
            lista_sentencas_inst[-1][1] += 1
        cont_tokens += 1 ## Incrementa um token ao contador, visto de ser um token considerado válido
        
lista_tokens_inst.append(cont_tokens - 1) ## Adiciona os token's calculados do id último id, visto que termina o for e não seria salvo na lista, o -1 é para retirar da contagem o token do número do id
lista_tokens_inst.pop(0) ## Retiro primeiro elemento da lista por ser um elemento sempre 0
lista_sentencas_inst[-1][1] -= 1 ## Reduz uma sentença visto que o o valor id é contado como sentença
lista_sentencas_inst.pop(0) ## Retiro primeiro elemento da lista por ser um elemento sem valor

[0, -1]

In [80]:
df_limpa = df_original.drop(axis=0, index=index_remove)

In [81]:
display(df_limpa)

,Palavras,Rotulo,Sentenca
3,NOME,O,2
4,DA,O,2
5,CACHAÇA,O,2
7,Cachaça,O,2
8,Três,B-NOME_BEBIDA,2
...,...,...,...
186013,e,O,14287
186014,Destilados,O,14287
186015,do,O,14287
186016,Brasil,B-NOME_LOCAL,14287


### Preparando / visualizando os dados para a estatística

In [82]:
lista_setenca_qtdTokens = list(df_limpa.groupby(by="Sentenca")["Palavras"].count()) ## Agrupa as sentenças conto quantos tokens tem por setença e transforma em lista
lista_sentencas_inst = [x[1] for x in lista_sentencas_inst]

In [83]:
print(lista_tokens_inst) ## imprimir a qtd de tokens por instância

[45, 244, 38, 99, 91, 165, 164, 44, 43, 55, 73, 79, 72, 22, 150, 48, 43, 78, 48, 45, 48, 84, 54, 46, 47, 99, 47, 39, 46, 37, 45, 37, 39, 26, 52, 50, 39, 42, 49, 84, 87, 94, 92, 49, 40, 58, 51, 59, 158, 46, 52, 94, 40, 34, 62, 56, 236, 334, 50, 47, 33, 95, 157, 123, 86, 38, 33, 86, 106, 391, 154, 90, 78, 78, 75, 231, 59, 115, 199, 250, 106, 182, 64, 115, 252, 133, 87, 373, 127, 88, 222, 99, 136, 255, 78, 39, 27, 412, 111, 69, 44, 27, 106, 23, 254, 50, 145, 68, 57, 54, 91, 102, 16, 71, 15, 14, 165, 58, 30, 183, 138, 220, 236, 121, 123, 313, 120, 66, 92, 15, 111, 77, 17, 17, 15, 155, 29, 104, 40, 293, 149, 118, 157, 191, 167, 23, 44, 21, 153, 59, 252, 81, 377, 131, 154, 62, 194, 21, 204, 55, 165, 41, 381, 46, 319, 126, 166, 62, 197, 186, 158, 25, 72, 150, 88, 378, 188, 191, 478, 48, 203, 66, 158, 123, 150, 65, 259, 456, 105, 366, 62, 93, 70, 259, 267, 98, 141, 58, 180, 346, 223, 226, 127, 275, 299, 299, 281, 199, 133, 130, 121, 133, 129, 160, 142, 142, 161, 163, 353, 353, 105, 212, 92, 93

In [87]:
print(lista_sentencas_inst)

[9, 20, 9, 10, 12, 17, 15, 9, 9, 9, 9, 11, 12, 4, 11, 8, 8, 10, 9, 9, 9, 9, 9, 7, 9, 12, 9, 9, 8, 8, 9, 8, 8, 6, 9, 9, 9, 8, 9, 9, 12, 10, 10, 8, 7, 9, 9, 9, 17, 9, 9, 9, 6, 7, 8, 9, 16, 22, 9, 9, 7, 11, 11, 13, 16, 6, 6, 7, 8, 19, 9, 8, 10, 7, 9, 17, 8, 11, 13, 14, 8, 11, 7, 7, 15, 7, 6, 15, 12, 5, 17, 9, 9, 13, 9, 5, 4, 19, 6, 8, 5, 4, 6, 4, 12, 5, 13, 5, 8, 4, 5, 11, 3, 5, 3, 3, 11, 6, 5, 12, 7, 15, 12, 11, 9, 15, 9, 6, 5, 3, 6, 6, 3, 3, 3, 8, 4, 10, 9, 27, 14, 15, 12, 18, 13, 4, 5, 4, 15, 11, 28, 13, 32, 17, 15, 14, 22, 4, 17, 8, 18, 4, 32, 9, 28, 13, 16, 10, 20, 18, 15, 4, 11, 18, 12, 32, 20, 17, 34, 9, 6, 4, 12, 7, 15, 3, 26, 35, 5, 24, 9, 12, 6, 26, 20, 5, 7, 9, 18, 26, 18, 11, 16, 18, 19, 19, 16, 17, 16, 16, 17, 17, 17, 17, 16, 16, 16, 17, 20, 20, 16, 11, 4, 4, 3, 3, 3, 8, 17, 23, 13, 19, 21, 20, 8, 20, 11, 8, 22, 10, 12, 10, 13, 14, 13, 20, 12, 18, 14, 8, 8, 5, 5, 23, 23, 23, 23, 23, 23, 23, 23, 23, 13, 13, 15, 14, 21, 12, 24, 24, 15, 15, 18, 9, 11, 9, 14, 14, 15, 15, 14, 13, 

## Informações da Estatística

In [88]:
print("Quantidade de tokens removidos: " + str(len(df_original) - len(df_limpa)))

Quantidade de tokens removidos: 29995


### Dados demográficos 

In [89]:
print("-----QUANTIDADES-----")
print("Quantidade de instâncias: " + str(len(lista_tokens_inst)))
print("Quantidade de sentenças: " + str(len(lista_setenca_qtdTokens)))
print("Quantidade de tokens: " + str(len(df_limpa)))

-----QUANTIDADES-----
Quantidade de instâncias: 1000
Quantidade de sentenças: 13273
Quantidade de tokens: 156024


### TOKENS POR SENTENÇA

In [90]:
print("-----QUANTIDADES-----")
print("Quantidade de sentenças: " + str(len(lista_setenca_qtdTokens)))
print("Quantidade de tokens: " + str(len(df_limpa)))
print("-----Médias-----")
print("Média aritmética dos tokens por sentença: " + str(statistics.mean(lista_setenca_qtdTokens)))
print("Média geométrica dos tokens por sentença: " + str(statistics.geometric_mean(lista_setenca_qtdTokens)))
print("Média harmônica dos tokens por sentença: " + str(statistics.harmonic_mean(lista_setenca_qtdTokens)))
print("-----Medianas-----")
print("Mediana dos tokens por setença: " + str(statistics.median(lista_setenca_qtdTokens)))
print("Mediana inferior dos tokens por setença: " + str(statistics.median_low(lista_setenca_qtdTokens)))
print("Mediana superior dos tokens por setença: " + str(statistics.median_high(lista_setenca_qtdTokens)))
print("Mediana, ou o 50º percentil dos dados agrupados dos tokens por setença: " + str(statistics.median_grouped(lista_setenca_qtdTokens)))
print("-----Moda-----")
print("Moda dos tokens por sentença: " + str(statistics.mode(lista_setenca_qtdTokens)))
print("-----Desvio Padrão-----")
print("Desvio padrão populacional dos tokens por sentença: " + str(statistics.pstdev(lista_setenca_qtdTokens)))
print("Desvio padrão amostral dos tokens por sentença: " + str(statistics.stdev(lista_setenca_qtdTokens)))
print("-----Variância-----")
print("Variância populacional dos tokens por sentença: " + str(statistics.pvariance(lista_setenca_qtdTokens)))
print("Variância amostral dos tokens por sentença: " + str(statistics.variance(lista_setenca_qtdTokens)))
print("-----Quantiles-----")
print("Quantiles dos tokens por sentença: " + str(statistics.quantiles(lista_setenca_qtdTokens)))

-----QUANTIDADES-----
Quantidade de sentenças: 13273
Quantidade de tokens: 156024
-----Médias-----
Média aritmética dos tokens por sentença: 11.75499133579447
Média geométrica dos tokens por sentença: 7.1176454709857095
Média harmônica dos tokens por sentença: 4.635240249172114
-----Medianas-----
Mediana dos tokens por setença: 6
Mediana inferior dos tokens por setença: 6
Mediana superior dos tokens por setença: 6
Mediana, ou o 50º percentil dos dados agrupados dos tokens por setença: 6.486780715396579
-----Moda-----
Moda dos tokens por sentença: 3
-----Desvio Padrão-----
Desvio padrão populacional dos tokens por sentença: 17.79746428391955
Desvio padrão amostral dos tokens por sentença: 17.798134760451042
-----Variância-----
Variância populacional dos tokens por sentença: 316.74973493739196
Variância amostral dos tokens por sentença: 316.7736009511757
-----Quantiles-----
Quantiles dos tokens por sentença: [3.0, 6.0, 15.0]


### TOKENS POR INSTÂNCIA

In [91]:
print("-----QUANTIDADES-----")
print("Quantidade de instâncias: " + str(len(lista_tokens_inst)))
print("Quantidade de tokens: " + str(len(df_limpa)))
print("-----Médias-----")
print("Média aritmética dos tokens por instância: " + str(statistics.mean(lista_tokens_inst)))
print("Média geométrica dos tokens por instância: " + str(statistics.geometric_mean(lista_tokens_inst)))
print("Média harmônica dos tokens por instância: " + str(statistics.harmonic_mean(lista_tokens_inst)))
print("-----Medianas-----")
print("Mediana dos tokens por instância: " + str(statistics.median(lista_tokens_inst)))
print("Mediana inferior dos tokens por instância: " + str(statistics.median_low(lista_tokens_inst)))
print("Mediana superior dos tokens por instância: " + str(statistics.median_high(lista_tokens_inst)))
print("Mediana, ou o 50º percentil dos dados agrupados dos tokens por instância: " + str(statistics.median_grouped(lista_tokens_inst)))
print("-----Moda-----")
print("Moda dos tokens por instância: " + str(statistics.mode(lista_tokens_inst)))
print("-----Desvio Padrão-----")
print("Desvio padrão populacional dos tokens por instância: " + str(statistics.pstdev(lista_tokens_inst)))
print("Desvio padrão amostral dos tokens por instância: " + str(statistics.stdev(lista_tokens_inst)))
print("-----Variância-----")
print("Variância populacional dos tokens por instância: " + str(statistics.pvariance(lista_tokens_inst)))
print("Variância amostral dos tokens por instância: " + str(statistics.variance(lista_tokens_inst)))
print("-----Quantiles-----")
print("Quantiles dos tokens por instância: " + str(statistics.quantiles(lista_tokens_inst)))

-----QUANTIDADES-----
Quantidade de instâncias: 1000
Quantidade de tokens: 156024
-----Médias-----
Média aritmética dos tokens por instância: 156.024
Média geométrica dos tokens por instância: 120.56283251415897
Média harmônica dos tokens por instância: 93.09696192162718
-----Medianas-----
Mediana dos tokens por instância: 119.0
Mediana inferior dos tokens por instância: 119
Mediana superior dos tokens por instância: 119
Mediana, ou o 50º percentil dos dados agrupados dos tokens por instância: 119.0
-----Moda-----
Moda dos tokens por instância: 99
-----Desvio Padrão-----
Desvio padrão populacional dos tokens por instância: 119.55740639542161
Desvio padrão amostral dos tokens por instância: 119.61722997004112
-----Variância-----
Variância populacional dos tokens por instância: 14293.973424
Variância amostral dos tokens por instância: 14308.281705705705
-----Quantiles-----
Quantiles dos tokens por instância: [71.0, 119.0, 198.75]


### SENTENÇAS POR INSTÂNCIA

In [92]:
print("-----QUANTIDADES-----")
print("Quantidade de instâncias: " + str(len(lista_tokens_inst)))
print("Quantidade de sentenças: " + str(len(lista_setenca_qtdTokens)))
print("-----Médias-----")
print("Média aritmética dos tokens por instância: " + str(statistics.mean(lista_sentencas_inst)))
print("Média geométrica dos tokens por instância: " + str(statistics.geometric_mean(lista_sentencas_inst)))
print("Média harmônica dos tokens por instância: " + str(statistics.harmonic_mean(lista_sentencas_inst)))
print("-----Medianas-----")
print("Mediana dos tokens por instância: " + str(statistics.median(lista_sentencas_inst)))
print("Mediana inferior dos tokens por instância: " + str(statistics.median_low(lista_sentencas_inst)))
print("Mediana superior dos tokens por instância: " + str(statistics.median_high(lista_sentencas_inst)))
print("Mediana, ou o 50º percentil dos dados agrupados dos tokens por instância: " + str(statistics.median_grouped(lista_sentencas_inst)))
print("-----Moda-----")
print("Moda dos tokens por instância: " + str(statistics.mode(lista_sentencas_inst)))
print("-----Desvio Padrão-----")
print("Desvio padrão populacional dos tokens por instância: " + str(statistics.pstdev(lista_sentencas_inst)))
print("Desvio padrão amostral dos tokens por instância: " + str(statistics.stdev(lista_sentencas_inst)))
print("-----Variância-----")
print("Variância populacional dos tokens por instância: " + str(statistics.pvariance(lista_sentencas_inst)))
print("Variância amostral dos tokens por instância: " + str(statistics.variance(lista_sentencas_inst)))
print("-----Quantiles-----")
print("Quantiles dos tokens por instância: " + str(statistics.quantiles(lista_sentencas_inst)))

-----QUANTIDADES-----
Quantidade de instâncias: 1000
Quantidade de sentenças: 13273
-----Médias-----
Média aritmética dos tokens por instância: 13.273
Média geométrica dos tokens por instância: 11.383559636372114
Média harmônica dos tokens por instância: 9.733068066169684
-----Medianas-----
Mediana dos tokens por instância: 11.0
Mediana inferior dos tokens por instância: 11
Mediana superior dos tokens por instância: 11
Mediana, ou o 50º percentil dos dados agrupados dos tokens por instância: 11.03623188405797
-----Moda-----
Moda dos tokens por instância: 9
-----Desvio Padrão-----
Desvio padrão populacional dos tokens por instância: 7.441268104295127
Desvio padrão amostral dos tokens por instância: 7.444991531150246
-----Variância-----
Variância populacional dos tokens por instância: 55.372471
Variância amostral dos tokens por instância: 55.427898898898896
-----Quantiles-----
Quantiles dos tokens por instância: [8.0, 11.0, 18.0]


# Testes / Exemplos / Demonstração

### Calculando desvio padrão

In [ ]:
print(df_limpa.groupby(by="Sentenca")["Palavras"].count().std())

### Calculando moda

In [ ]:
print(df_limpa.groupby(by="Sentenca")["Palavras"].count().mode())

### Calculando Mediana

In [ ]:
print(df_limpa.groupby(by="Sentenca")["Palavras"].count().median())

### Calculo da média com desvio padrão

In [ ]:
multiplicador = 1 ## Valor a ser alterado para validar em outros casos
valor = df_limpa.groupby(by="Sentenca")["Palavras"].count().mean() + multiplicador * df_limpa.groupby(by="Sentenca")["Palavras"].count().std()
print("A média mais o desvio padrão vezes " + str(multiplicador) + " resultou no valor: " + str(valor))
resultado = len(list(filter(lambda x : x > valor or x < 11, list(df_limpa.groupby(by="Sentenca")["Palavras"].count()))))
print("A quantidade de sentenças que possui mais de " + str(valor) + "tokens é de: " + str(resultado))
print("Que representa um total de " + str(100 - (resultado / len(df_limpa.groupby(by="Sentenca"))) * 100) + "% das setenças")

### Imprimindo as setenças que possuem apenas 1 token

In [ ]:
teste = [index for index, num in enumerate(df_limpa.groupby(by="Sentenca")["Palavras"].count()) if num == 1]
palavras_lista = list(df_limpa.groupby(by="Sentenca")["Palavras"])
print("Quantidade de sentenças com 1 token: " + str(len(teste)))
for index in teste:
    print(palavras_lista[index][1])

### Teste
Teste feito em ambiente menor.

In [ ]:
df_teste = df_limpa.loc[0:20]

In [ ]:
display(df_teste)

In [ ]:
print(list(df_teste.groupby(by="Sentenca")["Palavras"].count()))

In [ ]:
print(df_teste.groupby(by="Sentenca")["Palavras"].count().mean())

In [ ]:
print(df_teste.groupby(by="Sentenca")["Palavras"].count().std())

In [ ]:
print(df_limpa.groupby(by="Sentenca")["Palavras"].count().mode())

In [ ]:
print(df_limpa.groupby(by="Sentenca")["Palavras"].count().median())